In [11]:
import pandas as pd
import seaborn as sns
df = sns.load_dataset('iris')

## 타입

출처 : http://pbpython.com/pandas_dtypes.html

pandas는 기본 로딩 시 표와 같은 데이터 타입을 활용 // 최대값을 활용하기 때문에 비효율적 // 타입 변경이 필요

### 1. Numeric Type

#### int, float 데이터 타입의 경우, Numpy type으로 변경이 가능하다. ( 메모리 최적화 활용 )

In [34]:
df['sepal_length'] = df['sepal_length'].astype('uint8')
df['sepal_width'] = df['sepal_width'].astype('float16')

# df['컬럼명'].astype('타입명')

**to_numeric 활용**

errors='coerce'를 선택하면 에러없이 그 값은 Nan으로 바뀐다.

downcast는 4가지 중 선택하면 최소한의 데이터 타입으로 선택한다??

In [81]:
# errors : {'ignore', 'raise', 'coerce'}
# downcast : {'integer', 'signed', 'unsigned', 'float' }
df['petal_length'] = pd.to_numeric(df['petal_length'], errors='coerce', downcast='integer')

### 2. String Type

참고 : https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html

기존에는 문자열을 numpy가 사용하는 object 타입으로 처리했다.

1.0부터 string 타입이 새로 생겼다. 성능에는 아직 차이가 없다고 함

In [13]:
df['species'] = df['species'].astype('string')

In [14]:
df['species']

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: string

### 3. Category : 지정된 값으로만 채워질 수 있는 타입 

( 나머지는 NaN 처리 ) / 숫자, 문자 가능 / R에서 factor와 같음

*object 타입을 category로 바꾸면 메모리 절약에 큰 효과가 있다.* 단, unique한 값이 50% 이상이면 역효과

In [17]:
import pandas as pd
import seaborn as sns
df = sns.load_dataset('iris')

In [18]:
df['species'] = df['species'].astype('category')

In [19]:
df.species

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: category
Categories (3, object): [setosa, versicolor, virginica]

In [23]:
# 임의로 생성 예시
## categories가 지정할 값, 나머지는 NaN 처리
pd.Categorical(['a','b','c','d','e'], categories=['a','b'], ordered=True)

[a, b, NaN, NaN, NaN]
Categories (2, object): [a < b]

#### .cat + 변수 또는 함수 활용

참고 : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.cat.html

In [20]:
df['species'].cat.categories

Index(['setosa', 'versicolor', 'virginica'], dtype='object')

### 4. Time Type 

date 타입이 object 타입으로 로드되는 경우가 있을 때, 활용할 수 있다.

In [45]:
import pandas as pd

date_df = pd.DataFrame(['2019-01-01 01:00:00','2019-02-03 05:00:00'], columns=['date'])

In [46]:
date_df.dtypes # object 확인

date    object
dtype: object

format은 현재 데이터로 사용되는 형식을 입력한다. 성능 향상을 위해 반드시 입력하도록 한다.

In [47]:
## 현재 YYYY-mm-dd 이므로 %Y-%m-%d ( 반드시 %가 들어간 형식으로 )
date_df['date'] = pd.to_datetime(date_df['date'], format='%Y-%m-%d %H:%M:%S')

date type은 index컬럼으로 변경하여 활용하면 유용

특정 단위로 데이터 추출 가능

In [9]:
date_df['date'].dt.month

0    1
1    1
Name: date, dtype: int64

#### TIMEZONE이 들어간 데이터

**.dt.tz_localize** 또는 **.dt.tz_convert**를 활용할 수 있다.

In [70]:
import pandas as pd

date_df = pd.DataFrame(['2020-01-01 01:00:00','2020-02-03 05:00:00'], columns=['date'])

In [54]:
# tz_localize는 현재 시간의 TIMEZONE을 기록하는 의미이다.
## 현재 시간에 타임존을 명시하면 추후에 타임존을 변경할 수도 있다.
date_df['date'] = pd.to_datetime(date_df['date'], format='%Y-%m-%d %H:%M') \
                  .dt.tz_localize("Asia/Seoul")

# UTC의 경우, utc 옵션으로 바로 설정가능하다.
#date_df['date'] = pd.to_datetime(date_df['date'], format='%Y-%m-%d %H:%M', utc=True) 

In [55]:
date_df['date']

0   2020-01-01 01:00:00+09:00
1   2020-02-03 05:00:00+09:00
Name: date, dtype: datetime64[ns, Asia/Seoul]

In [56]:
# tz_convert를 사용하여 UTC로 변경
date_df['date'].dt.tz_convert("UTC")

0   2019-12-31 16:00:00+00:00
1   2020-02-02 20:00:00+00:00
Name: date, dtype: datetime64[ns, UTC]

데이터 자체에 TIMEZONE이 들어간 경우

In [63]:
import pandas as pd

date_df = pd.DataFrame(['2020-01-01 01:00:00 UTC','2020-02-03 05:00:00 UTC'], columns=['date'])

In [61]:
# %Z 가 TIMEZONE을 의미한다.
date_df['date'] = pd.to_datetime(date_df['date'], format='%Y-%m-%d %H:%M:%S %Z')

In [62]:
date_df['date']

0   2020-01-01 01:00:00+00:00
1   2020-02-03 05:00:00+00:00
Name: date, dtype: datetime64[ns, UTC]